# Sensitivity check for choice of impact functions

In this code, we (qualitatively) examine how sensitive modelling outcomes are to the choice of impact functions. For this, we generate 4 scenarios and test these for both hazard types (CF, TC) on 2 countries (PHL, SOM):

* base scenario: 'original' impact functions (about 15), distributed to types of buildings
* min scenario: assigning 4-5 base classes to our building types, taking a high vulnerability scenario for each of them
* med scenario: ditto, taking medium vuln. versions of impfs
* max scenario: ditto, taking low vuln. versions of impfs

In [1]:
# Load country BEM as prep
from climada.util.constants import SYSTEM_DIR
from climada.hazard import TropCyclone, Hazard
from climada.entity.exposures import Exposures
from climada.engine import ImpactCalc
import numpy as np
import os
import copy
from pathlib import Path

os.chdir('/cluster/project/climate/evelynm/global-displacement-risk') 
import exposure
import vulnerability
import coastal_flood_hazard

cntry_som = 'Somalia'
cntry_phl = 'Philippines'

In [2]:
gdf_bem_subcomps = exposure.gdf_from_bem_subcomps(cntry_som, opt='full')
gdf_bem_subcomps = gdf_bem_subcomps[gdf_bem_subcomps.valhum>0.001] # filter out rows with basically no population

exp_som = Exposures(gdf_bem_subcomps.copy())
exp_som.value_unit = 'Pop. count'
exp_som.gdf['longitude'] = exp_som.gdf.geometry.x
exp_som.gdf['latitude'] = exp_som.gdf.geometry.y
exp_som.gdf['value'] = 1 # each building type gets value 1

In [3]:
gdf_bem_subcomps = exposure.gdf_from_bem_subcomps(cntry_phl, opt='full')
gdf_bem_subcomps = gdf_bem_subcomps[gdf_bem_subcomps.valhum>0.1] # filter out rows with basically no population

exp_phl = Exposures(gdf_bem_subcomps.copy())
exp_phl.value_unit = 'Pop. count'
exp_phl.gdf['longitude'] = exp_phl.gdf.geometry.x
exp_phl.gdf['latitude'] = exp_phl.gdf.geometry.y
exp_phl.gdf['value'] = 1 # each building type gets value 1

## TC impact functions

### Base scenario

In [10]:
exp_som.gdf['impf_TC_base'] = exp_som.gdf['se_seismo'].map(vulnerability.DICT_PAGER_TCIMPF_HAZUS)

In [11]:
exp_phl.gdf['impf_TC_base'] = exp_phl.gdf['se_seismo'].map(vulnerability.DICT_PAGER_TCIMPF_HAZUS)

### Min, med, max scenarios

In [22]:
# we modify the dictionary to point to the "_min", "_med" and "_max" vulnerability functions everywhere 
DICT_TCIMPF_MIN = {key: vulnerability.DICT_HAZUS_IMPF_TC_HAZUS[value[:-3]+'min']
                           for key, value in vulnerability.DICT_PAGER2HAZUS.items()}
DICT_TCIMPF_MED = {key: vulnerability.DICT_HAZUS_IMPF_TC_HAZUS[value[:-3]+'med']
                           for key, value in vulnerability.DICT_PAGER2HAZUS.items()}
DICT_TCIMPF_MAX = {key: vulnerability.DICT_HAZUS_IMPF_TC_HAZUS[value[:-3]+'max']
                           for key, value in vulnerability.DICT_PAGER2HAZUS.items()}

In [25]:
exp_som.gdf['impf_TC_min'] = exp_som.gdf['se_seismo'].map(DICT_TCIMPF_MIN) 
exp_phl.gdf['impf_TC_min'] = exp_phl.gdf['se_seismo'].map(DICT_TCIMPF_MIN) 

In [26]:
exp_som.gdf['impf_TC_med'] = exp_som.gdf['se_seismo'].map(DICT_TCIMPF_MED) 
exp_phl.gdf['impf_TC_med'] = exp_phl.gdf['se_seismo'].map(DICT_TCIMPF_MED) 

In [27]:
exp_som.gdf['impf_TC_max'] = exp_som.gdf['se_seismo'].map(DICT_TCIMPF_MAX) 
exp_phl.gdf['impf_TC_max'] = exp_phl.gdf['se_seismo'].map(DICT_TCIMPF_MAX) 

### Perform impact calcs and compare

In [1]:
# load TC hazard phl
haz_TC_dir = Path('/cluster/work/climate/meilers/hazard/present')
haz_TC_phl = TropCyclone.from_hdf5(haz_TC_dir.joinpath('TC_WP_0150as_MIT_H08.hdf5'))
# load TC hazard som
haz_TC_dir = Path('/cluster/work/climate/meilers/hazard/present')
haz_TC_som = TropCyclone.from_hdf5(haz_TC_dir.joinpath('TC_IO_0150as_MIT_H08.hdf5'))

NameError: name 'TropCyclone' is not defined

In [ ]:
TC_phl = haz_TC_phl.select(extent=(
    exp_phl.gdf['longitude'].min(), exp_phl.gdf['longitude'].max(), exp_phl.gdf['latitude'].min(), exp_phl.gdf['latitude'].max())

In [ ]:
TC_som = haz_TC_som.select(extent=(
    exp_som.gdf['longitude'].min(), exp_som.gdf['longitude'].max(), exp_som.gdf['latitude'].min(), exp_som.gdf['latitude'].max())

In [ ]:
exp_som.gdf.rename({'impf_TC_base' : 'impf_TC'}, axis=1, inplace=True)
exp_phl.gdf.rename({'impf_TC_base' : 'impf_TC'}, axis=1, inplace=True)

imp_som_base = ImpactCalc(exp_som, vulnerability.IMPF_SET_TC_HAZUS, TC_som).impact()
imp_phl_base = ImpactCalc(exp_som, vulnerability.IMPF_SET_TC_HAZUS, TC_phl).impact()

In [32]:
exp_som.gdf.rename({'impf_TC' : 'impf_TC_base'}, axis=1, inplace=True) # change column name back
exp_phl.gdf.rename({'impf_TC' : 'impf_TC_base'}, axis=1, inplace=True) # change column name back
exp_som.gdf.rename({'impf_TC_min' : 'impf_TC'}, axis=1, inplace=True)
exp_phl.gdf.rename({'impf_TC_min' : 'impf_TC'}, axis=1, inplace=True)

imp_som_min = ImpactCalc(exp_som, vulnerability.IMPF_SET_TC_HAZUS, TC_som).impact()
imp_phl_min = ImpactCalc(exp_som, vulnerability.IMPF_SET_TC_HAZUS, TC_phl).impact()

In [ ]:
exp_som.gdf.rename({'impf_TC' : 'impf_TC_min'}, axis=1, inplace=True) # change column name back
exp_phl.gdf.rename({'impf_TC' : 'impf_TC_min'}, axis=1, inplace=True) # change column name back
exp_som.gdf.rename({'impf_TC_med' : 'impf_TC'}, axis=1, inplace=True)
exp_phl.gdf.rename({'impf_TC_med' : 'impf_TC'}, axis=1, inplace=True)

imp_som_med = ImpactCalc(exp_som, vulnerability.IMPF_SET_TC_HAZUS, TC_som).impact()
imp_phl_med = ImpactCalc(exp_som, vulnerability.IMPF_SET_TC_HAZUS, TC_phl).impact()

In [ ]:
exp_som.gdf.rename({'impf_TC' : 'impf_TC_med'}, axis=1, inplace=True) # change column name back
exp_phl.gdf.rename({'impf_TC' : 'impf_TC_med'}, axis=1, inplace=True) # change column name back
exp_som.gdf.rename({'impf_TC_max' : 'impf_TC'}, axis=1, inplace=True)
exp_phl.gdf.rename({'impf_TC_max' : 'impf_TC'}, axis=1, inplace=True)

imp_som_max = ImpactCalc(exp_som, vulnerability.IMPF_SET_TC_HAZUS, TC_som).impact()
imp_phl_max = ImpactCalc(exp_som, vulnerability.IMPF_SET_TC_HAZUS, TC_phl).impact()

In [ ]:
print(imp_som_base.aai_agg, imp_som_min.aai_agg, imp_som_med.aai_agg, imp_som_max.aai_agg)

In [ ]:
print(imp_phl_base.aai_agg, imp_phl_min.aai_agg, imp_phl_med.aai_agg, imp_phl_max.aai_agg)

## FL impact functions

### Base Scenario

In [34]:
exp_som.gdf['impf_FL_base'] = exp_som.gdf['se_seismo'].map(vulnerability.DICT_PAGER_FLIMPF_CIMA)
exp_phl.gdf['impf_FL_base'] = exp_phl.gdf['se_seismo'].map(vulnerability.DICT_PAGER_FLIMPF_CIMA)

### Min, med, max scenarios

In [38]:
# manual mapping from vulnerable impf-ids to less vulnerable impf-ids and vice versa (referring do CIMA funcs)
DICT_FLIMPF_MIN = {1:1,2:1,3:1,4:1,5:1,6:6,7:6,8:8,9:8,10:8,11:11,12:12,13:12,14:14,15:15}
DICT_FLIMPF_MED = {1:2, 2:2, 3:2,4:2,5:2,6:7,7:7, 8:7,9:7,10:7,11:11,12:13,13:13,14:15,15:15}
DICT_FLIMPF_MAX = {1:4, 2:4, 3:4, 4:4, 5:5, 6:10, 7:10, 8:10, 9:10, 10:10, 11:11, 12:13,13:13,14:15,15:15}    

In [39]:
exp_som.gdf['impf_FL_min'] = exp_som.gdf['impf_FL_base'].map(DICT_FLIMPF_MIN) 
exp_phl.gdf['impf_FL_min'] = exp_phl.gdf['impf_FL_base'].map(DICT_FLIMPF_MIN) 

In [40]:
exp_som.gdf['impf_FL_med'] = exp_som.gdf['impf_FL_base'].map(DICT_FLIMPF_MED) 
exp_phl.gdf['impf_FL_med'] = exp_phl.gdf['impf_FL_base'].map(DICT_FLIMPF_MED)

In [41]:
exp_som.gdf['impf_FL_max'] = exp_som.gdf['impf_FL_base'].map(DICT_FLIMPF_MAX) 
exp_phl.gdf['impf_FL_max'] = exp_phl.gdf['impf_FL_base'].map(DICT_FLIMPF_MAX) 

### Perform impact calcs and compare

In [ ]:
haz_CF_dir = Path('/cluster/work/climate/evelynm/IDMC_UNU/hazard/coastal_flood/venDEM_scaled_1km')

In [ ]:
# get tiles covering the exposure extent
tiles_phl = coastal_flood_hazard.find_tiles(
    exp_phl.gdf['latitude'].min(), exp_phl.gdf['latitude'].max(), exp_phl.gdf['longitude'].min(), exp_phl.gdf['longitude'].max())
tiles_som = coastal_flood_hazard.find_tiles(
    exp_som.gdf['latitude'].min(), exp_som.gdf['latitude'].max(), exp_som.gdf['longitude'].min(), exp_som.gdf['longitude'].max())

In [ ]:
# load CF hazard phl
CF_phl = coastal_flood_hazard.generate_hazard_object(tiles_phl, haz_CF_dir, 'RCP45', '2020', 'FL')
# load CF hazard som
CF_som = coastal_flood_hazard.generate_hazard_object(tiles_som, haz_CF_dir, 'RCP45', '2020', 'FL')

In [ ]:
exp_som.gdf.rename({'impf_FL_base' : 'impf_FL'}, axis=1, inplace=True)
exp_phl.gdf.rename({'impf_FL_base' : 'impf_FL'}, axis=1, inplace=True)

imp_som_base = ImpactCalc(exp_som, vulnerability.IMPF_SET_FL_CIMA, CF_som).impact()
imp_phl_base = ImpactCalc(exp_som, vulnerability.IMPF_SET_FL_CIMA, CF_phl).impact()

In [32]:
exp_som.gdf.rename({'impf_FL' : 'impf_FL_base'}, axis=1, inplace=True) # change column name back
exp_phl.gdf.rename({'impf_FL' : 'impf_FL_base'}, axis=1, inplace=True) # change column name back
exp_som.gdf.rename({'impf_TC_min' : 'impf_FL'}, axis=1, inplace=True)
exp_phl.gdf.rename({'impf_TC_min' : 'impf_FL'}, axis=1, inplace=True)

imp_som_min = ImpactCalc(exp_som, vulnerability.IMPF_SET_FL_CIMA, CF_som).impact()
imp_phl_min = ImpactCalc(exp_som, vulnerability.IMPF_SET_FL_CIMA, CF_phl).impact()

In [ ]:
exp_som.gdf.rename({'impf_FL' : 'impf_TC_min'}, axis=1, inplace=True) # change column name back
exp_phl.gdf.rename({'impf_FL' : 'impf_TC_min'}, axis=1, inplace=True) # change column name back
exp_som.gdf.rename({'impf_FL_med' : 'impf_FL'}, axis=1, inplace=True)
exp_phl.gdf.rename({'impf_FL_med' : 'impf_FL'}, axis=1, inplace=True)

imp_som_med = ImpactCalc(exp_som, vulnerability.IMPF_SET_FL_CIMA, CF_som).impact()
imp_phl_med = ImpactCalc(exp_som, vulnerability.IMPF_SET_FL_CIMA, CF_phl).impact()

In [ ]:
exp_som.gdf.rename({'impf_FL' : 'impf_FL_med'}, axis=1, inplace=True) # change column name back
exp_phl.gdf.rename({'impf_FL' : 'impf_FL_med'}, axis=1, inplace=True) # change column name back
exp_som.gdf.rename({'impf_FL_max' : 'impf_FL'}, axis=1, inplace=True)
exp_phl.gdf.rename({'impf_FL_max' : 'impf_FL'}, axis=1, inplace=True)

imp_som_max = ImpactCalc(exp_som, vulnerability.IMPF_SET_FL_CIMA, CF_som).impact()
imp_phl_max = ImpactCalc(exp_som, vulnerability.IMPF_SET_FL_CIMA, CF_phl).impact()

In [ ]:
print(imp_som_base.aai_agg, imp_som_min.aai_agg, imp_som_med.aai_agg, imp_som_max.aai_agg)

In [ ]:
print(imp_phl_base.aai_agg, imp_phl_min.aai_agg, imp_phl_med.aai_agg, imp_phl_max.aai_agg)